# QuickStart

_AI application framework that makes function calling with LLM easier_.

---

This demonstrates how to seamlessly integrate any Python function into your LLM application using ActionWeaver and either the Azure or OpenAI API.
 

In [ ]:
%load_ext autoreload
%autoreload 2

**Use ActionWeaver and OpenAI API**

In [ ]:
from actionweaver.utils.tokens import TokenUsageTracker
from actionweaver.llms import wrap
from openai import OpenAI

openai_client = wrap(OpenAI())

In [ ]:
from actionweaver import action
@action(name="GetCurrentTime")
def get_current_time() -> str:
    """
    Use this for getting the current time in the specified time zone.
    
    :return: A string representing the current time in the specified time zone.
    """
    print ("Getting current time...")
    import datetime
    current_time = datetime.datetime.now()
    
    return f"The current time is {current_time}"


@action(name="GetWeather", stop=False)
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    print ("Getting current weather")
    
    import json
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what's the weather in San Francisco and Beijing ?"}
  ]

response = openai_client.create(
  model="gpt-3.5-turbo",
  messages=messages,
    actions = [get_current_weather],
    stream=False, 
    token_usage_tracker = TokenUsageTracker(500),
)

response

**Use ActionWeaver and Azure OpenAI Service**

In [ ]:
import os
from openai import AzureOpenAI

azure_client = wrap(AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2023-10-01-preview"
))

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "what's the weather in San Francisco and Beijing ?"}
  ]

response = azure_client.create(
  model="gpt-4-32k",
  messages=messages,
  stream=False,
  actions = [get_current_weather],
  token_usage_tracker = TokenUsageTracker(500),
)

response

**Easily integrate tools from libraries such as [Langchain](https://github.com/langchain-ai/langchain/tree/master/libs/community)**

In [ ]:
from langchain_community.tools.google_search.tool import GoogleSearchRun
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
search_tool = GoogleSearchRun(api_wrapper=GoogleSearchAPIWrapper())

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "search what is ActionWeaver on Github"}
  ]

from actionweaver.actions.factories.langchain import action_from_tool

response = azure_client.create(
  model="gpt-35-turbo-0613-16k",
  messages=messages,
  stream=False,
  actions = [action_from_tool(search_tool, description="Invoke this tool to search any information")],
  token_usage_tracker = TokenUsageTracker(500),
)

response

**Force execute an action**

You can also compel the language model to execute the action by calling the invoke method of an action. Its arguments includes the ActionWeaver-wrapped client and other arguments passed to the create API.

In [ ]:
get_current_time.invoke(openai_client, messages=[{"role": "user", "content": "what time"}], model="gpt-3.5-turbo", stream=False, force=False)


In [ ]:
get_current_time.invoke(azure_client, messages=[{"role": "user", "content": "what time"}], model="gpt-35-turbo-0613-16k", stream=False, force=False)


**Stop the Action in the loop**

Every action comes with a stop argument, which is set to False by default, if True this means that the LLM will immediately return the function's output if chosen, but this also restricts the LLM from making multiple function calls. For instance, if asked about the weather in NYC and San Francisco, the model would invoke two separate functions sequentially for each city. However, with `stop=True`, this process is interrupted once the first function returns weather information for either NYC or San Francisco, depending on which city it queries first.

In [ ]:
get_current_weather.stop = True

In [ ]:
get_current_weather.invoke(openai_client, messages=[{"role": "user", "content": "what weather is San Francisco"}], model="gpt-3.5-turbo", stream=False, force=True)


**Actions of Stateful Agent**

Developers also could create a class and enhance its functionality using ActionWeaver's action decorators.

In [ ]:
class AgentV0:
    def __init__(self):
        self.llm = wrap(OpenAI())
        self.messages = []
        self.times = []
    
    def __call__(self, text):
        self.messages += [{"role": "user", "content":text}]
        return self.llm.create(model="gpt-3.5-turbo", messages=self.messages, actions = [self.get_current_time])
        
    @action(name="GetCurrentTime")
    def get_current_time(self) -> str:
        """
        Use this for getting the current time in the specified time zone.
        
        :return: A string representing the current time in the specified time zone.
        """
        import datetime
        current_time = datetime.datetime.now()

        self.times += [str(current_time)]
        
        return f"The current time is {current_time}"

agent = AgentV0()

agent("what time is it") # Output: 'The current time is 20:34.'

# You can invoke actions just like regular instance methods
agent.get_current_time() # Output: 'The current time is 20:34.'

**Grouping and Extending Actions Through Inheritance**

In this example, we wrap the LangChain Google search as a method, and define a new agent that inherits the previous agent and LangChain search tool. This approach leverages object-oriented principles to enable rapid development and easy expansion of the agent's capabilities.

In the example below, through inheritance, the new agent can utilize the Google search tool method as well as any other actions defined in the parent classes.

In [ ]:
class LangChainTools:
    @action(name="GoogleSearch")
    def google_search(self, query: str) -> str:
        """
        Perform a Google search using the provided query. 
        
        This action requires `langchain` and `google-api-python-client` installed, and GOOGLE_API_KEY, GOOGLE_CSE_ID environment variables.
        See https://python.langchain.com/docs/integrations/tools/google_search.

        :param query: The search query to be used for the Google search.
        :return: The search results as a string.
        """

        search = GoogleSearchAPIWrapper()
        return search.run(query)
    
class AgentV1(AgentV0, LangChainTools):
    def __call__(self, text):
        self.messages += [{"role": "user", "content":text}]
        return self.llm.create(model="gpt-3.5-turbo", messages=self.messages, actions = [self.google_search])

agent = AgentV1()
agent("what happened today")

"""
Output: Here are some events that happened or are scheduled for today (August 23, 2023):\n\n1. Agreement State Event: Event Number 56678 - Maine Radiation Control Program.\n2. Childbirth Class - August 23, 2023, at 6:00 pm.\n3. No events scheduled for August 23, 2023, at Ambassador.\n4. Fine Arts - Late Start.\n5. Millersville University events.\n6. Regular City Council Meeting - August 23, 2023, at 10:00 AM.\n\nPlease note that these are just a few examples, and there may be other events happening as well.
"""